<h1 align="center">Software Introspection for Signaling Emergent Cyber-Social Operations (SIGNAL)</h1>
<h2 align="center">SRI International</h2>
<h3 align="center">In support of DARPA AIE Hybrid AI to Protect Integrity of Open Source Code (SocialCyber)</h3>

<h2 align="center">dev2vec: A multi-task learning approach for understanding inter-and-intra developer interactions in open-source software development</h2>

## Introduction

Open-source communities strive to evaluate contributed code on its technical correctness
and merits. However, these processes are usually loosely supervised and are often
influenced by additional factors, including here but not limited to, social rules, trust,
reputation, and arcane processes. These additional components lead to *unexpected*
opportunities for subversion. Current approaches, designed to help software development
teams to manage the code assessment process, determine the trustworthiness of OSS based on
publicly available traces involving code, revision history, logs, and external packages.
Although useful, these approaches tend to disregard important inter-and-intra developer
interactions that could drastically affect an OSS project's life-cycle. 

In this notebook, we
introduce a systematic approach and a deep multi-task learning architecture, termed
**dev2vec**, for identifying and understanding these developer interactions. We
empirically evaluate our approach and deep multi-task learning architecture on the Linux
Kernel, a mature ecosystem with a vibrant community and a wealth of socio-technical
developer interactions, utilizing conversations and interactions originating from its
Linux Kernel Mailing List (LKML). Our approach successfully identifies key developer
activities determining the distinct roles developers assume in the LKML. 


## Background

In this section, we motivate the need for understanding the inter-and-intra developer in
large OSS projects, particularly in the context of case study: the Linux Kernel and its
mailing list.

The Linux Kernel(LK) [1] is a mature
open-source software with a vibrant community. In the last couple of decades, it has grown
and evolved into a large open-source ecosystem. As an ecosystem, it spawns a sufficiently
\textit{diverse set of social and technical interactions} through the involvement of
heterogenous community in its Linux Kernel Mailing List (LKML) [2]. It has a hierarchy of mature sub-systems, each of these sub-systems have history
of developer interactions and thus can be used *to evidence common behavioral
patterns*. For these reasons, we have chosen the LK as our case study. With this case
study, we seek to understand how contributions are carried out over time and thus capture
common behavioral patterns.

Motivated by these considerations, our work builds upon a general work in Activity-based
analysis of OSS projects [3], specializes it to LK case study, and
then extends it with a deep multi-task learning architecture to model developer behavior and
thus capture role dynamism over a user-specified time window. The latter is inspired by
work in user simulation using recurrent neural networks [4]. In this user
simulation work, recurrent neural networks are applied to describing users in a given
domain without expert supervision [5].

To establish this understanding of developer actions, roles, and these roles dynamism, we
first collect and curate data from the LKML within a user-specified time window. These
data contains key attributes concerning communication and interactions among LK
developers. We then transform them to make sure we can learn a function that maps a sequence
of past developer activity as input to a series of observations, within the specified
window. This transformation effort aims capturing concrete actions taken by LK developers from
different perspectives, which have grown and evolved over time. In the Table below, we enlist the transformed data.

| Metric             | Description                                                                  |
|--------------------|------------------------------------------------------------------------------|
| message_exper      | # of patch emails sent by a developer in earlier email threads.              |
| commit_exper.      | # of committed patches by a developer.                                       |
| fkre_score.        | Avg. Flesch Kincaid Reading Ease score (text comprehension difficulty).      |
| fkgl_score.        | Avg. Flesch Kincaid Grade Level Score (text reading grade level).            |
| verbosity.         | Wordiness level of a patch email (ratio of # words against # sentences).     |
| exert_persuasion   | Measures whether influence has been exerted via persuasion strategies.       |
| sent_time          | Year/month/week of the year/Day of week in which a patch email was sent.     |
| received_time      | Year/month/week of the year/Day of week in which a patch email was received. |
| reply_within_4hrs  | Measures whether sent out patch email receives a reply email within 4 hrs.   |
| patch_email        | Measures whether an email a patch email.                                     |
| first_patch_thread | Measures whether a patch email initiated an email thread.                    |
| patch_churn        | Measures whether a patch in email is a patch rewrite (update).               |
| bug_fix            | Measures whether the patch in email is a bug fix patch.                      |
| new_feature        | Measures whether the patch in email is a new feature patch.                  |
| accepted_patch     | Measures whether patch an email is an accepted patch in LKML.                |
| accepted_commit    | Measures whether patch in email is a committed patch in LK.                  |

A preliminary analysis on the transformed data indicated that the collected metrics are
inter-related and thus can be determined by a set of *latent factors*. Based on
this finding, we hypothesize that *these latent factors are the typical activities
that LK developers engage in when assuming certain roles in the LKML*. To help us
understand these latent inter-relations, we perform factor
analysis [6]. Factor analysis is an exploratory data analysis
method that we can use for both *extracting* latent factors from our
data and simplifying (i.e., *rotating*) the structure of these data to
improve their interpretability. Then, we use its output to identify the typical activities in
the LKML.

Using these typical activities, we design a deep multi-task learning architecture for
modeling the dynamic nature of activities in the LKML. Multi-task learning (MTL) is a
subfield of machine learning where a model attempts to concurrently learn a series of
*related* tasks [7, 8, 9], e.g., simultaneously learning the evaluate the distance, spin, and
trajectory of a ball in a ping-pong game [10, 11].
Note that each tasks has it's own loss function. In the context of MTL, we seek to
minimize the sum of the loss functions [9]. MTL is well-suited
for the problem at hand, given that we are interested in asking a series of questions
related to future developer activity.

## dev2vec

We now move on to  introducing ***dev2vec***, our LSTM-based multi-task learning model designed to learn from a sequence of past developer activities and output a series of observations corresponding to the upcoming (future) developer actions. Note that, we use the term *dev2vec* interchangeably referring to the data representation or the deep MTL architecture, depending on the context.

![dev2vec architecture](figures/dev2vec.png)

The design of *dev2vec*'s architecture, Figure above, is inspired by the work of Zolna et
al. [5]. In this work, the authors use an Long-Short-Term Memory
(LSTM)-based MTL architecture to describe to describe the characteristics of users
involved in Real-Time Bidding online auction events. Similarly, we aim at having a model
capable of answering questions of interest regarding future developer actions.

Our deep MTL model is sequential in structure. It takes as input a sequence of vectors
composed of $n$-floating point numbers in the $[-1, 1]$ range. Here, the term $n$ refers
to the number of features depicting developer characteristics that we extracted through
factor analysis. The input is fed to an LSTM layer with 300 memory cells, applying a
dropout of $0.15$. Differently from *user2vec*, our second LSTM layer also contains
300-memory cells. The output of the last LSTM layer is then fed as input to 3-different
fully connected (FC) layers, or *heads*, as commonly referred in the literature. Each
FC model is trained to answer to one of the following questions of interest:

1. Will the next action be a triage action?
2. Will the next action result in a small code change?
3. Will the next action result in a controversial code change, sparking a lot of discussions?

We bring to the attention of the reader that the architecture can be easily extended to accommodate different tasks of interest.

#### References

[1] The Linux Kernel. https://github.com/torvalds/linux

[2] The Linux Kernel Mailing List. https://lkml.org/

[3] Cheng, Jinghui, and Jin LC Guo. "*Activity-based analysis of open source software contributors: Roles and dynamics.*" In 2019 IEEE/ACM 12th International Workshop on Cooperative and Human Aspects of Software Engineering (CHASE), pp. 11-18. IEEE, 2019. [PDF](https://arxiv.org/pdf/1903.05277.pdf)

[4] Gür, Izzeddin, Dilek Hakkani-Tür, Gokhan Tür, and Pararth Shah. "*User modeling for task oriented dialogues.*" In 2018 IEEE Spoken Language Technology Workshop (SLT), pp. 900-906. IEEE, 2018. [PDF](https://arxiv.org/pdf/1811.04369.pdf)

[5] Żołna, Konrad, and Bartłomiej Romański. "*User modeling using LSTM networks.*" In Thirty-First AAAI Conference on Artificial Intelligence. 2017. [PDF](https://www.aaai.org/ocs/index.php/AAAI/AAAI17/paper/viewFile/14220/14254)

[6] Child, Dennis. *The essentials of factor analysis*. Cassell Educational, 1990.

[7] Crawshaw, Michael. "Multi-task learning with deep neural networks: A survey." arXiv preprint arXiv:2009.09796 (2020). [PDF](https://arxiv.org/pdf/2009.09796.pdf)

[8] Standley, Trevor, Amir Zamir, Dawn Chen, Leonidas Guibas, Jitendra Malik, and Silvio Savarese. "*Which tasks should be learned together in multi-task learning?.*" In International Conference on Machine Learning, pp. 9120-9132. PMLR, 2020. [PDF](http://proceedings.mlr.press/v119/standley20a/standley20a.pdf)

[9] Soni, Devin. Multi-task learning in Machine Learning. https://towardsdatascience.com/multi-task-learning-in-machine-learning-20a37c796c9c

[10] Fifty, Christopher. Deciding Which Tasks Should Train Together in Multi-Task Neural Networks. https://ai.googleblog.com/2021/10/deciding-which-tasks-should-train.html

[11] Fifty, Chris, Ehsan Amid, Zhe Zhao, Tianhe Yu, Rohan Anil, and Chelsea Finn. "*Efficiently identifying task groupings for multi-task learning.*" Advances in Neural Information Processing Systems 34 (2021). [PDF](https://proceedings.neurips.cc/paper/2021/file/e77910ebb93b511588557806310f78f1-Paper.pdf)


#### Disclaimer

The content of this notebook is released under the **GNU General Public License v3.0**, see [LICENSE](https://github.com/SRI-CSL/signal-public/blob/main/LICENSE).